# Diophantine equations

A Diophantine equation is polynomial with integers for both variable coefficients and solutions.

| Problem Type  | Solution Domain | Example |
|---------------|-----------------|---------|
| Diophantine   | Integers only   | x = 2   |
| Linear system | Real numbers    | x = 1.5 |

The standard form of a matrix is: `Ax = b`
* `A` = coefficient matrix (m × n)
* `x` = variables (n × 1) — what you're solving for
* `b` = constants (m × 1) — the targets
* `m` = number of equations
* `n` = number of variables

For instance, from Advent of Code 2025 Day 10 we have a sample machine defined as:

```
(2) (0,2) (1,2) (2,3) {14,146,179,10}
```

The first four tuples `()` are buttons we can press (variables) to increment the wired counters by +1, and the final set `{}` contains the target values for those counters (constants).

If we press the second button `(0,2)` it will increment both counter 0 and counter 2 by +1 each.

We are asked for the minimum number of button presses required to reach the target values.

We can construct a matrix as follows:

```
        b0  b1  b2  b3  |  constants
c0:      0   1   0   0  |  14
c1:      0   0   1   0  |  146
c2:      1   1   1   1  |  179
c3:      0   0   0   1  |  10
```

The left-hand side is the coefficient matrix, and the right-hand side is the constant vector.

For each column (button) of the coefficient matrix we use `1` to represent if that button is linked to each counter's row, and `0` otherwise.

We can see several facts at a glance. For example:
* Button 0 only affects counter 2
* Counter 2 is affected by every button
* Counter 2 must be incremented exactly 179 times

For certain matrices we can solve the equations by hand.

Ideally, we want a **square** matrix with a **full rank**.

| Condition | Name            | Meaning                       |
|-----------|-----------------|-------------------------------|
| m = n     | Square          | Equal equations and variables |
| m > n     | Overdetermined  | More equations than variables |
| m < n     | Underdetermined | More variables than equations |

| Rank           | Meaning               | Number of solutions                               |
|----------------|-----------------------|---------------------------------------------------|
| Full rank      | Distinct constraints  | One                                               |
| Rank-deficient | Redundant constraints | Zero (inconsistent) or Infinite (underdetermined) |

A **full rank** matrix has no redundant constraints, nor constraints that are a combination of others.

This matrix is rank-deficient because it has a duplicate constraint (we can omit):

```
        b0  b1  b2  b3  b4  b5
c0:      0   1   0   1   0   1   = 10
c1:      0   1   0   1   0   1   = 10   ← same constraint!
c2:      1   0   0   0   0   0   = 5
...
```

And this matrix is rank-deficient because it contains a linear combination:

```
c0:      1   1   0   0   = 10
c1:      0   0   1   1   = 10
c2:      1   1   1   1   = 20   ← this is just c0 + c1!
```

A rank can never exceed the smaller dimension of the matrix.

| Matrix Shape | Maximum Rank |
|--------------|--------------|
| 6 x 6        | 6            |
| 8 x 5        | 5            |
| 3 x 7        | 3            |

Our example matrix from above is both square and full rank:

```
        b0  b1  b2  b3  |  constants
c0:      0   1   0   0  |  14
c1:      0   0   1   0  |  146
c2:      1   1   1   1  |  179
c3:      0   0   0   1  |  10
```

Because `c0`, `c1`, and `c3` are each only influenced by a single button, we set the value of those buttons to each counter's constant.

| Variable | Value |
|----------|-------|
| b0       |       |
| b1       | 14    |
| b2       | 146   |
| b3       | 10    |

We're now only missing the solution of `c2`, which is:

c2: `b0 + b1 + b2 + b3 = 179`

We can substitute the known values of `b1`, `b2`, and `b3`:

c2: `b0 + 14 + 146 + 10 = 179`

We combine like terms:

c2: `b0 + 170 = 179`

And then subtract `170` from both sides to isolate `b0`:

c2: `b0 = 9`

We press each button this many times:

| Button     | Presses |
|------------|---------|
| b0         | 9       |
| b1         | 14      |
| b2         | 146     |
| b3         | 10      |

For a total of **179** button presses.

Fun fact: This example could have also been solved by realizing that if every button press increments counter `c2`, its target constant is the solution.

# Informal Gaussian Elimination

Let's look at a more complicated example. This machine has seven buttons and nine counters:

```
(2,3) (1,4,6) (0,1,3,4,5,6,8) (0,1,3,4,7) (0,7) (0,2,3,4,5,6,8) (4,5,8) {199,32,11,30,53,39,38,174,39}
```

Its matrix would be:

```
        b0  b1  b2  b3  b4  b5  b6  |  target
c0:      0   0   1   1   1   1   0  |  199
c1:      0   1   1   1   0   0   0  |  32
c2:      1   0   0   0   0   1   0  |  11
c3:      1   0   1   1   0   1   0  |  30
c4:      0   1   1   1   0   1   1  |  53
c5:      0   0   1   0   0   1   1  |  39
c6:      0   1   1   0   0   1   0  |  38
c7:      0   0   0   1   1   0   0  |  174
c8:      0   0   1   0   0   1   1  |  39
```

We don't have any equations only influenced by a single button:

| Counter | Equation                 | Target |
|---------|--------------------------|--------|
| c0      | `b2 + b3 + b4 + b5`      | 199    |
| c1      | `b1 + b2 + b3`           | 32     |
| c2      | `b0 + b5`                | 11     |
| c3      | `b0 + b2 + b3 + b5`      | 30     |
| c4      | `b1 + b2 + b3 + b5 + b6` | 53     |
| c5      | `b2 + b5 + b6`           | 39     |
| c6      | `b1 + b2 + b5`           | 38     |
| c7      | `b3 + b4`                | 174    |
| c8      | `b2 + b5 + b6`           | 39     |

Notice that `c5` and `c8` are identical, so we can drop `c8` and are left with only eight equations.

Now `c2` is fully contained within `c3`, so we can subtract them:

```
c3:   b0 + (b2 + b3) + b5 =  30
c2: - b0             - b5 = -11
-------------------------------
           (b2 + b3)      =  19
```

Whenever we see the pattern `b2 + b3` we can substitute it with `19`.

| Variable | Value |
|----------|-------|
| b2 + b3  | 19    |

One example is `c1`:

```
c1: b1 + (b2 + b3) = 32
    b1 + 19        = 32
-----------------------
       - 19         -19
-----------------------
    b1             = 13
```

We've solved our first variable, and can now substitute the constant `13` for `b1`.

Let's substitute `b1` in equation `c6`:

```
c6: (b1) + b2 + b5 = 38
    13   + b2 + b5 = 38
-----------------------
  - 13             - 13
-----------------------
           b2 + b5 = 25
```

We now know that `b2 + b5` is `25`:

| Variable | Value |
|----------|-------|
| b1       | 13    |
| b2 + b3  | 19    |
| b2 + b5  | 25    |

We can substitute that in `c5`:

```
c5: (b2 + b5) + b6 = 39
           25 + b6 = 39
-----------------------
         - 25      - 25
-----------------------
                b6 = 14
```

We've solved our second variable, and can now use `14` in place of `b6`.

| Variable | Value |
|----------|-------|
| b0       |       |
| b1       | 13    |
| b2       |       |
| b3       |       |
| b4       |       |
| b5       |       |
| b6       | 14    |
| b2 + b3  | 19    |
| b2 + b5  | 25    |

Our list of equations looks like:

| Counter | Equation                              | Target |
|---------|---------------------------------------|--------|
| c0      | `(b2+b3=19) + b4 + b5`                | 199    |
| c1      | `(b1=13) + (b2+b3=19)`                | 32     |
| c2      | `b0 + b5`                             | 11     |
| c3      | `b0 + (b2+b3=19) + b5`                | 30     |
| c4      | `(b1=13) + (b2+b3=19) + b5 + (b6=14)` | 53     |
| c5      | `(b2+b5=25) + (b6=14)`                | 39     |
| c6      | `(b1=13) + (b2+b5=25)`                | 38     |
| c7      | `b3 + b4`                             | 174    |

We can solve for `b5` with equation `c4`:

```
c4: 13 + 19 + b5 + 14 = 53
              b5 - 46 = 53 - 46
              b5      = 7
```

| Counter | Equation                                  | Target |
|---------|-------------------------------------------|--------|
| c0      | `(b2+b3=19) + b4 + (b5=7)`                | 199    |
| c1      | `(b1=13) + (b2+b3=19)`                    | 32     |
| c2      | `b0 + (b5=7)`                             | 11     |
| c3      | `b0 + (b2+b3=19) + (b5=7)`                | 30     |
| c4      | `(b1=13) + (b2+b3=19) + (b5=7) + (b6=14)` | 53     |
| c5      | `(b2+b5=25) + (b6=14)`                    | 39     |
| c6      | `(b1=13) + (b2+b5=25)`                    | 38     |
| c7      | `b3 + b4`                                 | 174    |

We can solve for `b0` with equation `c2` (`b0 = 11-7 = 4`) or `c3` (`b0 = 30-7-19 = 4`).

And solve for `b4` with equation `c0` (`b4 = 199 - 19 - 7 = 173`).

And solve for `b3` with equation `c7` (`b3 = 174 - 173 = 1`).

And finally solve for `b2` with `b2 + b3 = 19` (`b2 = 19 - 1 = 18`).

| Variable | Value |
|----------|-------|
| b0       | 4     |
| b1       | 13    |
| b2       | 18    |
| b3       | 1     |
| b4       | 173   |
| b5       | 7     |
| b6       | 14    |

We'd press **230** buttons to reach our target counters.

| Counter | Equation                                  | Target |
|---------|-------------------------------------------|--------|
| c0      | `(b2+b3=19) + (b4=173) + (b5=7)`          | 199    |
| c1      | `(b1=13) + (b2+b3=19)`                    | 32     |
| c2      | `(b0=4) + (b5=7)`                         | 11     |
| c3      | `(b0=4) + (b2+b3=19) + (b5=7)`            | 30     |
| c4      | `(b1=13) + (b2+b3=19) + (b5=7) + (b6=14)` | 53     |
| c5      | `(b2+b5=25) + (b6=14)`                    | 39     |
| c6      | `(b1=13) + (b2+b5=25)`                    | 38     |
| c7      | `(b3=1) + (b4=173)`                       | 174    |

This is the optimal solution.